# DATA INGESTION PIPELINE

In [1]:
import os

os.chdir("../") # currently in ../research so I'm trying to change the working directory to the whole project

In [2]:
%pwd # project working directory

'/Users/nanakwamekankam/Desktop/NK/Portfolio Projects/End-to-end-ML-Project-with-MLFlow'

In [3]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_url: str
    local_data_file: Path
    unzip_dir: Path

In [4]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories

In [5]:
class ConfigurationManager:
    def __init__(self):
        self,
        # config_filepath = CONFIG_FILE_PATH
        # param_filepath = PARAMS_FILE_PATH
        # schema_filepath = SCHEMA_FILE_PATH

        self.config = read_yaml(CONFIG_FILE_PATH)
        self.param = read_yaml(PARAMS_FILE_PATH)
        self.schema = read_yaml(SCHEMA_FILE_PATH)

        create_directories([self.config.artifacts_root])


    def get_data_ingestion_config(self):
        config = self.config.data_ingestion
        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig( 
            root_dir = config.root_dir,
            source_url = config.source_url,
            local_data_file = config.local_data_file,
            unzip_dir= config.unzip_dir
        )

        return data_ingestion_config

In [8]:
import os
import urllib.request as request
import zipfile
from mlProject import logger
from mlProject.utils.common import get_size


class DataIngestion:
    def __init__(self, config):
        self.config = config

    def download_data(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url= self.config.source_url,
                filename= self.config.local_data_file
            )
            
            logger.info(f"Downloaded data in {filename}. INFO: {headers}, SIZE: {get_size(Path(self.config.local_data_file))}")
        else:
            logger.info(f"File already exists. SIZE: {get_size(Path(self.config.local_data_file))}")

    
    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        if not os.path.exists(unzip_path):
            os.makedirs(unzip_path)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [10]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_data()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-02-03 16:19:26,811: INFO: common: yaml file: config/config.yaml loaded successfully]
[2025-02-03 16:19:26,814: INFO: common: yaml file: params.yaml loaded successfully]
[2025-02-03 16:19:26,815: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-02-03 16:19:26,816: INFO: common: created directory at: artifacts]
[2025-02-03 16:19:26,817: INFO: common: created directory at: artifacts/data_ingestion]
[2025-02-03 16:19:26,818: INFO: 3592443810: File already exists. SIZE: ~ 23 KB]
